In [47]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
import time
device = 'cuda' if torch.cuda.is_available() else 'cpu'
block_size = 8
batch_size = 4

In [48]:
with open('wizard_of_oz.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()
chars = sorted(set(text))
vocab_size = len(chars)

In [63]:
string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: "".join([int_to_string[i] for i in l])


data = torch.tensor(encode(text), dtype = torch.long)
print(len(data))

232309


In [59]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i+1: i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)

    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [51]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets = None):
        
        logits = self.token_embedding_table(index)
        
        if targets is not None:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            
            targets = targets.view(B*T)
            
            loss = F.cross_entropy(logits, targets)
            
            return logits, loss
        else: 
            loss = None
            return logits, loss
    
    def generate(self, index, max_new_tokens):
        
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:,-1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat([index, index_next], dim = 1)
        
        return index

In [52]:
model = BigramLanguageModel(vocab_size)
m = model.to(device)
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_char = decode(m.generate(context, max_new_tokens=500)[0].tolist())

generated_char

'\n3;tO9&4jzgzGw(M\n x*h-hzIpyd\ufeffx"AHDNOE2Wj*B?P6s:fd\')QDGoO3fSTyr_9,bb39rq7fb-AZKFp?\nFqDx)?6)r5W2g& (YGt6K[8*u&q"36ghywnqQDJtJ\ufeffhw&"ynw65d(ZI.fgW)KRpLW U\'\nPVTquH26.kfzm,fuHwj*sx5mj1?Tt)ti]r(Ue(se:oDgzpPmiAhaT:let3i1?x(??e4eeNW3]x:l:(QSp\n.\n**UNX)q4Rb[ev3pHE5X)\'qRDvq_ntPa\ufeffg);5GFqC4KbrkL?OE PutNB"Q6!nuEp9JGOBOTy4RFZ0OB_obAGlhiZ3;\ufeffXol!RF(4VrX]!mi:e7X]i(;hv*m90cFK,4BOlfb_AzNgo2p5mHsHcsRfMlmSZih9K[TH*9wFKouY c-h;6K*"Rvf)tkfF\'xPutdJj9 sXE4HoevWzg-3p.EC-4MPjbGAy:P*2F_Vcg8Hw\nPgVMCp55?KXlscy,;t_A0AMPlecOR;f][('

In [62]:
max_iters = 1000
learning_rate = 3e-4
eval_iters = 250
dropout = 0.2

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for i in range(max_iters):
    if i % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {i}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")
    
    xb, yb = get_batch('train')
    
    logits, loss = model.forward(xb, yb)
    
    optimizer.zero_grad(set_to_none= True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 3.073, val loss: 3.098
step: 250, train loss: 3.031, val loss: 3.082
step: 500, train loss: 3.003, val loss: 3.044
step: 750, train loss: 3.006, val loss: 3.034
3.3169021606445312


In [ ]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_char = decode(m.generate(context, max_new_tokens=500)[0].tolist())

generated_char